In [301]:
import pandas as pd
import json
from datetime import datetime, timezone,date
from atproto import Client
from ebird.api import get_nearby_notable,get_totals,get_notable_observations, get_observations, get_visits,get_checklist, get_species_observations
from dotenv import load_dotenv
import os
import requests

In [129]:
from PIL import Image
from io import BytesIO

In [417]:
import unicodedata

In [210]:
import time

In [326]:
from inspect import getclasstree

In [17]:
today = datetime.today().strftime('%Y-%m-%d')

In [9]:
load_dotenv()

True

In [77]:
bird_key = os.getenv("bird_key")
nuthatch_key = os.getenv("nuthatch")
bsky_username = os.getenv("bsky_username")
bsky_password = os.getenv("bsky_password")

In [15]:
client = Client()
client.login(bsky_username, bsky_password)

ProfileViewDetailed(did='did:plc:ct7g3cvr2jqqtdxlkdorqqft', handle='ny-state-birds.bsky.social', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:ct7g3cvr2jqqtdxlkdorqqft/bafkreiahaypxmz5hd7suxzdjjxdqjiakz7wkqcxwga5llnnkq6cp6hpb2i@jpeg', banner=None, created_at='2024-12-31T16:42:17.744Z', description=None, display_name='', followers_count=1, follows_count=1, indexed_at='2024-12-31T16:42:17.744Z', joined_via_starter_pack=None, labels=[], pinned_post=None, posts_count=0, viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewDetailed')

In [336]:
def find_local_notables(lat, lon, dist,bird_key):
    notables = get_nearby_notable(bird_key,lat, lon, dist,back=1)
    notables_df = pd.DataFrame(notables)
    notables_df = notables_df.drop_duplicates(subset=['comName'])
    return notables_df

In [331]:
notables = get_nearby_notable(bird_key, 40.01, -74.00, dist=50,back=1)
notables_df = pd.DataFrame(notables)

In [335]:
notables_df['obsDt'].min()

'2024-12-30 16:43'

In [330]:
notables_df['obsDt'] = pd.to_datetime(notables_df['obsDt'])  
today = datetime.today().date()

In [178]:
today_observations = notables_df[notables_df['obsDt'].dt.date == today]

In [346]:
local_notables = find_local_notables(40.01, -74.00,50, bird_key)

In [400]:
short_df = local_notables.sample(3)
short_df

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId
2,yetwar,Yellow-throated Warbler,Setophaga dominica,L15446407,Lakehurst--Willow Lake at Leisure Village West,2024-12-31 13:35,1,40.018682,-74.274010,False,False,False,S207359810
0,eurwig,Eurasian Wigeon,Mareca penelope,L298359,Lake of the Lilies,2024-12-31 14:59,1,40.083024,-74.043197,False,False,False,S207368014
18,laugul,Laughing Gull,Leucophaeus atricilla,L161004,Barnegat Lighthouse SP,2024-12-31 09:44,2,39.763333,-74.105833,False,False,False,S207372652


In [401]:
b_list = short_df['comName'].tolist()

In [ ]:
def make_list(df,num=3):
    df = df.head(num)
    b_list = df['comName'].tolist()
    return b_list

In [431]:
url = 'https://nuthatch.lastelm.software/v2/birds'


params = {
    'page': 1,
    'pageSize': 1,
    'name': 'yellow-throated warbler',
    'operator': 'AND'
}


headers = {
    'accept': 'application/json',
    'API-Key': nuthatch_key
}



In [432]:
response = requests.get(url, headers=headers, params=params)

In [433]:
response.json()

{'entities': [{'images': [],
   'lengthMin': '13',
   'lengthMax': '14',
   'name': 'Yellow-throated Warbler',
   'id': 609,
   'sciName': 'Setophaga dominica',
   'region': ['North America'],
   'family': 'Parulidae',
   'order': 'Passeriformes',
   'status': 'Low Concern'}],
 'total': 1,
 'page': 1,
 'pageSize': 1}

In [363]:
response.json()
data = response.json()
entity = data['entities'][0]
sci_name = entity['sciName']
images = entity['images']  
image_1 = images[0]
wingspan_max = entity['wingspanMax']
family = entity['family']
order = entity['order']
status = entity['status']
region = entity['region'][0] 

In [366]:
image_1

'https://images.unsplash.com/photo-1518570931581-f1d6f674af1b'

In [341]:
print(family,order,sci_name)

Picidae Piciformes Dryocopus pileatus


In [ ]:
response = requests.get(url, headers=headers, params=params)

In [376]:
one = local_notables.sample(1)

In [383]:
one

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId
8,bkcchi,Black-capped Chickadee,Poecile atricapillus,L197353,Sandy Hook,2024-12-31 11:48,2,40.439252,-73.986926,False,False,False,S207367318


In [388]:
def get_bird_info(df,url,headers):
    name = df['comName'].unique()[0]
    locname = df['locName'].unique()[0]
    cl_key = df['subId'].unique()[0]
    params = {
    'page': 1,
    'pageSize': 1,
    'name': name,
    'operator': 'AND'
    }   
    response = requests.get(url, headers=headers, params=params)

    response.json()
    data = response.json()
    entity = data['entities'][0]
    sci_name = entity['sciName']
    images = entity['images']  
    if images:
        image_1 = images[0]
    else:
        image_1 = "No image available"
    status = entity['status']
    family = entity['family']
    order = entity['order']
    region = entity['region'][0] 

    return sci_name, name, image_1, status, region,locname,family,order,cl_key

In [379]:
sci_name,name,image,concern,region,locname = get_bird_info(one,url,headers)

In [427]:
def write_nyc_text(df,url,headers):
    sci_name,name,image,concern,region,locname,family,order,cl_id = get_bird_info(df,url,headers)
    text = f"Notable bird near NYC! The {name}, also known as the {sci_name}. It was spotted at {locname}. An image of the {name} is below, if available. Here is a link to the eBird checklist https://ebird.org/checklist/{cl_id}"
    return text,image,name

In [424]:
a,b,c = write_nyc_text(local_notables,url,headers)

In [420]:
def count_graphemes(text):
    return sum(1 for _ in unicodedata.normalize('NFC', text))

In [425]:
count_graphemes(a)

250

In [218]:
def post(df, url, headers, max_width=800, max_height=600):
    text, image_url, name = write_nyc_text(df, url, headers)
    response = requests.get(image_url)
    response.raise_for_status()  
    image = Image.open(BytesIO(response.content))
    image.thumbnail((max_width, max_height)) 
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='JPEG') 
    img_byte_arr = img_byte_arr.getvalue()
    client.send_image(text=text, image=img_byte_arr, image_alt=name)

    print("post made")


In [394]:
def post(df, url, headers, max_width=800, max_height=600):
    text, image_url, name = write_nyc_text(df, url, headers)
    
    if image_url and image_url != "No image available":  
        try:
            response = requests.get(image_url)
            response.raise_for_status()  
            image = Image.open(BytesIO(response.content))
            image.thumbnail((max_width, max_height)) 
            img_byte_arr = BytesIO()
            image.save(img_byte_arr, format='JPEG') 
            img_byte_arr = img_byte_arr.getvalue()
            
            client.send_image(text=text, image=img_byte_arr, image_alt=name)
        except Exception as e:
            print(f"Failed to process the image: {e}")
            client.send_post(text=text)
    else:
        client.send_post(text=text)
    
    print("Post made")


In [131]:
post(local_notables,url,headers)

postt made


In [219]:
for bird in b_list:
    df = short_df[short_df['comName'] == bird]
    post(df,url,headers)
    time.sleep(5)

post made
post made
post made


In [404]:
def post_list(short_df,url,headers):
    for bird in b_list:
        df = short_df[short_df['comName'] == bird]
        post(df,url,headers)
        time.sleep(5)

In [402]:
for bird in b_list:
    print(bird)

Yellow-throated Warbler
Eurasian Wigeon
Laughing Gull


In [426]:
post_list(short_df,url,headers)

Post made
Post made
Post made


In [286]:
state_records = get_observations(bird_key, 'US-NY-047')

In [287]:
state_records_df = pd.DataFrame(state_records)

In [288]:
state_records_df['obsDt'] = pd.to_datetime(state_records_df['obsDt'])
today_state = state_records_df[state_records_df['obsDt'].dt.date == today]
today_state.shape

(69, 14)

In [302]:
today_state.head(2)

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId,exoticCategory
0,moudov,Mourning Dove,Zenaida macroura,L39034518,"175 Lefferts Ave, New York US-NY 40.66228, -73...",2024-12-31 12:29:00,15,40.662278,-73.955432,True,False,True,S207342141,NaN
1,ribgul,Ring-billed Gull,Larus delawarensis,L39034518,"175 Lefferts Ave, New York US-NY 40.66228, -73...",2024-12-31 12:29:00,1,40.662278,-73.955432,True,False,True,S207342141,NaN


In [303]:
brooklyn_bird_list = today_state['speciesCode'].tolist()

In [295]:
checklist = get_checklist(bird_key, 'S207342141')

In [284]:
totals = get_totals(bird_key, 'US-NY-047', date.today())

In [310]:
records = get_species_observations(bird_key, 'moudov', 'US-NY-047')
records_df = pd.DataFrame(records)
records_df.groupby('comName')['howMany'].sum().sort_values(ascending=False).reset_index()

,comName,howMany
0,Mourning Dove,497


In [313]:
def get_counts(bird_list):
    day_count = pd.DataFrame()
    for bird in bird_list:
        records = get_species_observations(bird_key, bird, 'US-NY-047')
        
        records_df = pd.DataFrame(records)
        summarized_df = (
            records_df.groupby('comName')['howMany']
            .sum()
            .sort_values(ascending=False)
            .reset_index()
        )
        day_count = pd.concat([day_count, summarized_df], ignore_index=True)
    
    return day_count

In [314]:
day_count = get_counts(brooklyn_bird_list)

In [317]:
day_count.sort_values('howMany',ascending=False).head(15)

,comName,howMany
36,Brant,8131.0
7,Canada Goose,3925.0
1,Ring-billed Gull,2289.0
12,Rock Pigeon,2214.0
15,American Herring Gull,1402.0
13,European Starling,1174.0
2,House Sparrow,708.0
42,Greater Scaup,701.0
41,Red-breasted Merganser,667.0
29,Ruddy Duck,612.0
